In [2]:
import random 
import time


def csp_backtracking(state):
    if (goal_test(state)):
        return state
    var = get_next_unassigned_var(state)
    for val in get_sorted_values(state, var):
        new_state = state.copy()
        new_state[var] = val
        result = csp_backtracking(new_state)
        if (result is not None):
            return result
    return None
def goal_test(state):
    if (None not in state):
        return True
            
def get_next_unassigned_var(state):
    for i in range(len(state)):
        if ((state[i] == None)):
            return i
    return None
def get_sorted_values(state, var): # discard any conflict in spaces
    available = []
    for i in range(len(state)):
        available.append(i)
    for i in range(len(state)):
        if (state[i] != None): 
            if (state[i] in available):
                available.remove(state[i])
            if (state[i] + abs(i-var) in available):
                available.remove(state[i] + abs(i-var))
            if (state[i] - abs(i-var) in available):
                available.remove(state[i] - abs(i-var))
    random.shuffle(available)
    return available

def test_solution(state):
    for var in range(len(state)):
        left = state[var]
        middle = state[var]
        right = state[var]
        for compare in range(var + 1, len(state)):
            left -= 1
            right += 1
            if state[compare] == middle:
                print(var, "middle", compare)
                return False
            if left >= 0 and state[compare] == left:
                print(var, "left", compare)
                return False
            if right < len(state) and state[compare] == right:
                print(var, "right", compare)
                return False
    return True

def incremental_repair(state):
    generate_board(state)
    num, conflicts = count_conflicts2(state)
    while (num > 0):
        maxi = max(conflicts)
        index = conflicts.index(maxi)
        if (conflicts.count(maxi) > 1):
            indices = [i for i, e in enumerate(conflicts) if e == maxi]
            random.shuffle(indices)
            index = indices[0]
        space = count_queen2(state, index)
        state[index] = space
        num, conflicts = count_conflicts2(state)
    return state

def generate_board(state):
    i = 0
    space = 1
    while (None in state):
        state[i] = space
        space += 3
        space = space % len(state)
        i+=1
    return state

def count_conflicts(state):
    conflict = 0
    conflicts = [0]*len(state)
    for var in range(len(state)):
        left = state[var]
        middle = state[var]
        right = state[var]
        leftCon = True
        rightCon = True
        middleCon = True
        for compare in range(var + 1, len(state)):
            left -= 1
            right += 1
            if state[compare] == middle:
                conflict += 1
                if (middleCon):
                    conflicts[compare] += 1
                    conflicts[var] += 1
                    middleCon = False
            if left >= 0 and state[compare] == left:
                conflict += 1
                if (leftCon):
                    conflicts[compare] += 1
                    conflicts[var] += 1
                    leftCon = False
            if right < len(state) and state[compare] == right:
                conflict += 1
                if (rightCon):
                    conflicts[compare] += 1
                    conflicts[var] += 1
                    rightCon = False
    return conflict, conflicts

def count_conflicts2(state):
    conflict = 0
    conflicts = [0]*len(state)
    rows = [0]*len(state)
    for i in range(len(state)):
        rows[state[i]] += 1
    for i in range(len(state)):
        n = rows[state[i]] - 1
        conflicts[i] = n
    n = len(state)
    diagonal = [0]*(((n-1)*2)+1)
    #left diagonal
    for i in range(len(state)):
        diagonal[state[i] + (n-1-i)] += 1
    for i in range(len(state)):
        conflicts[i] += diagonal[state[i] + (n-1-i)] - 1
    
    #right diagonal
    diagonal2 = [0]*(((n-1)*2)+1)
    for i in range(len(state)):
        diagonal2[state[i] + i] += 1
    for i in range(len(state)):
        conflicts[i] += diagonal2[state[i] + i] - 1
        
    conflict = sum(conflicts)/2    
    return conflict, conflicts


def count_queen(state, n):
    mini = len(state)
    m = 0
    positions = []
    for i in range(len(state)):
        state[n] = i
        a, b, c = True, True, True
        conflict = 0
        for j in range(n+1, len(state)):
            f = j-n
            if (state[j] == i and a == True):
                a = False
                conflict += 1
            elif (state[j] + f == i and b == True):
                b = False
                conflict += 1
            elif (state[j] - f == i and c == True):
                c = False
                conflict += 1
        a, b, c = True, True, True
        for r in range(0, n):
            f = n-r
            if (state[r] == i and a == True):
                a = False
                conflict += 1
            elif (state[r] + f == i and b == True):
                b = False
                conflict += 1
            elif (state[r] - f == i and c == True):
                c = False
                conflict += 1
        if (conflict <= mini):
            if (random.random() > 0.5):
                mini = conflict
                m = i
            if (mini == len(state)):
                mini = conflict
                m = i  
    return m, positions

def count_queen2(state, n):
    conflicts = [0]*len(state)
    positions = []
    for i in range(len(state)):
        conflicts[state[i]] += 1
        r = abs(i-n)
        if ((state[i] + r) < len(state)):
            conflicts[state[i] + r] += 1
        if ((state[i] - r) >= 0):
            conflicts[state[i] - r] += 1
    mini = min(conflicts)
    for i in range(len(conflicts)):
        if (conflicts[i] == mini):
            positions.append(i)       
    random.shuffle(positions)
    return positions[random.randint(0, len(positions)-1)]
        




start = time.perf_counter()
solutionList = []
n = 8

while (n <= 200):
    board = [None] * n
    solution = incremental_repair(board)
    solutionList.append(solution)
    n += 1
    end = time.perf_counter()
    if (end-start > 30):
        break
    
valid = True
for i in solutionList:
    if (test_solution(i) == False):
       valid = False

if valid:
    print("All solutions verified with test_solution()")
else:
    print("Solutions are NOT all verified")

All solutions verified with test_solution()


In [3]:
solutionList[0]

[3, 5, 7, 1, 6, 0, 2, 4]

In [4]:
# [3, 5, 7, 1, 6, 0, 2, 4] -> Queen column position in i-th row (so col 3 in row 0)
print(solutionList)

[[3, 5, 7, 1, 6, 0, 2, 4], [1, 4, 7, 0, 8, 5, 2, 6, 3], [2, 5, 8, 0, 3, 6, 9, 1, 4, 7], [1, 4, 7, 10, 2, 5, 8, 0, 3, 6, 9], [4, 7, 3, 8, 11, 2, 0, 5, 1, 9, 6, 10], [1, 4, 7, 10, 0, 3, 6, 9, 12, 2, 5, 8, 11], [8, 4, 7, 10, 13, 1, 6, 0, 3, 11, 9, 12, 2, 5], [6, 2, 7, 5, 13, 9, 14, 0, 3, 12, 8, 11, 4, 10, 1], [1, 5, 7, 10, 13, 0, 6, 3, 14, 12, 8, 4, 2, 15, 11, 9], [1, 4, 7, 10, 13, 16, 2, 5, 8, 11, 14, 0, 3, 6, 9, 12, 15], [14, 4, 7, 0, 13, 16, 6, 8, 2, 15, 3, 9, 11, 17, 5, 10, 12, 1], [1, 4, 7, 10, 13, 16, 0, 3, 6, 9, 12, 15, 18, 2, 5, 8, 11, 14, 17], [4, 10, 7, 17, 19, 1, 9, 18, 0, 8, 16, 11, 6, 14, 2, 13, 3, 12, 15, 5], [18, 4, 12, 8, 15, 17, 0, 2, 5, 7, 10, 13, 16, 19, 1, 14, 9, 6, 3, 20, 11], [0, 14, 7, 19, 13, 16, 4, 17, 3, 5, 9, 12, 15, 20, 2, 21, 18, 8, 11, 1, 6, 10], [1, 4, 7, 10, 13, 16, 19, 22, 2, 5, 8, 11, 14, 17, 20, 0, 3, 6, 9, 12, 15, 18, 21], [0, 18, 4, 10, 13, 11, 17, 21, 6, 2, 20, 14, 3, 9, 15, 23, 5, 1, 22, 7, 12, 16, 19, 8], [1, 4, 7, 10, 13, 16, 19, 22, 0, 3, 6, 9, 12